In [ ]:
# Imports

# Internal
import db_helpers as dbh
import db_parse as dbp

# External
import ipywidgets as widgets
import json
import pandas as pd
from pprint import pprint
import os

In [ ]:
# Settings

db_name = "dos"
schema = "schema.sql"
data_dir = "/usr/local/dos-mitigation/data"

In [ ]:
# Select Materialization

subfolders = [ f.path for f in os.scandir(data_dir) if f.is_dir() ]
materializations = [i.split('/')[-1] for i in subfolders]
widget_materialization = widgets.Dropdown(
    options=materializations,
    value=materializations[0],
    description='Materialization',
    disabled=False
)
widget_materialization

In [ ]:
# Select Session

materialization = widget_materialization.value
revision = materialization
materialization_dir = data_dir + '/' + materialization
subfolders = [ f.path for f in os.scandir(materialization_dir) if f.is_dir() ]
sessions = [i.split('/')[-1] for i in subfolders]
widget_session = widgets.Dropdown(
    options=sessions,
    value=sessions[0],
    description='Session',
    disabled=False
)
widget_session

In [ ]:
# Check for Existing Data

session = widget_session.value
session_dir = materialization_dir + '/' + session
conn = dbh.Connection("dos")

existing_revisions = conn.db_query("select distinct nickname from revisions")[0]
existing_materializations = conn.db_query("select distinct nickname from materializations")[0]
existing_sessions = conn.db_query("select distinct nickname from sessions")[0]

# if revision in existing_revisions:
#     print(f'WARNING: Revision {revision} already exists')
# if materialization in existing_materializations:
#     print(f'WARNING: Materialization {materialization} already exists')

widget_existing_session = None
if session in existing_sessions:
    print(f'WARNING: Session {session} already exists')
    widget_existing_session = widgets.Dropdown(
        options=['Append Nickname', 'Merge and Keep New Parameters', 'Merge and Keep Old Parameters'],
        value = 'Append Nickname',
        description = 'Action:',
        disabled=False
    )
widget_existing_session


In [ ]:
# Populate/Retrieve Metadata

if revision not in existing_revisions:
    conn.insert_dict_as_row('revisions', {
        "hash": "5bcc425177d073587dc0f78bf9acf084676d2061",
        "metal": True,
        "model_file": None,
        "nickname": revision
    })
revision_id = conn.nickname_id("revisions", revision)
    
if materialization not in existing_materializations:
    conn.insert_dict_as_row('materializations', {
        "nickname": materialization,
        "revision": revision_id,
        "inventory_file": None
    })
materialization_id = conn.nickname_id("materializations", materialization)

if widget_existing_session != None:
    existing_session_action = widget_existing_session.value
    if existing_session_action == 'Append Nickname':
        i = 0
        while session + str(i) in existing_sessions:
            i += 1
        print(f'Appending session nickname from {session} to {session + str(i)}')
        with open("{}/.parameters.json".format(session_dir)) as f:
            parameters = json.load(f)
        conn.insert_dict_as_row("sessions", {
            "materialization": materialization_id,
            "parameters": parameters,
            "nickname": session + str(i)
        })

    elif existing_session_action == 'Merge and Keep New Parameters':
        session_nick = session
        with open("{}/.parameters.json".format(session_dir)) as f:
            parameters = json.load(f)
        conn.insert_dict_as_row("sessions", {
            "materialization": materialization_id,
            "parameters": parameters,
            "nickname": session
        })
        
    else:
        pass

session_id = conn.nickname_id("sessions", session)

In [ ]:
# Parse Session Experiments

experiment_list = os.listdir("{}".format(session_dir))
experiment_list = [i for i in experiment_list if not i.startswith('.')]
print(f"Found {len(experiment_list)} experiments")
for experiment in experiment_list:
    if experiment.startswith('.'):
        continue
    dbp.parse_experiment(conn, materialization, session, experiment)
    dbp.analyze_experiment(conn, experiment)